# Optimization Results Database
### View and compare all experiment runs

In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd

print("database viewer loaded")

database viewer loaded


## Load Results from Database

In [2]:
# load all experiments
try:
    with open('results_database.json', 'r') as f:
        results = json.load(f)
    print(f"loaded {len(results)} experiment(s)")
except FileNotFoundError:
    print("no database found yet - run optimization first")
    results = []
except json.JSONDecodeError:
    print("database corrupted")
    results = []

loaded 3 experiment(s)


In [3]:
# create summary table
if results:
    data = {
        'exp #': list(range(1, len(results)+1)),
        'date': [datetime.fromisoformat(r['timestamp']).strftime('%m/%d %H:%M') for r in results],
        'f(x)': [r['f_value'] for r in results],
        'evals': [r['evaluations'] for r in results],
        'time(s)': [r['time_seconds'] for r in results],
        'seed': [r['seed'] for r in results],
        'comment': [r['comment'][:40] for r in results]
    }
    
    df = pd.DataFrame(data)
    
    # format f(x) column for scientific notation
    df['f(x)'] = df['f(x)'].apply(lambda x: f'{x:.6e}')
    
    print(df.to_string(index=False))
    
    # highlight best
    best_idx = np.argmin([r['f_value'] for r in results])
    print(f"\nbest experiment: #{best_idx+1} with f(x) = {results[best_idx]['f_value']:.12e}")
else:
    print("no experiments to display")

KeyError: 'time_seconds'

## Statistics

In [ ]:
# calculate statistics
if results:
    f_vals = np.array([r['f_value'] for r in results])
    
    print("=" * 50)
    print("STATISTICS")
    print("=" * 50)
    
    print(f"\ntotal experiments: {len(results)}")
    print(f"best f(x):  {np.min(f_vals):.15e}")
    print(f"worst f(x): {np.max(f_vals):.15e}")
    print(f"mean f(x):  {np.mean(f_vals):.15e}")
    print(f"std f(x):   {np.std(f_vals):.15e}")
    
    # trend analysis
    if len(results) >= 4:
        recent = np.mean(f_vals[-3:])
        older = np.mean(f_vals[:3])
        improvement = (older - recent) / older * 100
        
        print(f"\ntrend: ", end='')
        if improvement > 0:
            print(f"improving ({improvement:.1f}% better)")
        else:
            print(f"degrading ({abs(improvement):.1f}% worse)")

## Visualizations

In [ ]:
# create comprehensive visualization
if not results:
    print("no results to plot")
else:
    fig = plt.figure(figsize=(16, 10))
    
    # extract data
    f_vals = [r['f_value'] for r in results]
    dates = [datetime.fromisoformat(r['timestamp']) for r in results]
    evals = [r['evaluations'] for r in results]
    times = [r['time_seconds'] for r in results]
    comments = [r['comment'] for r in results]
    
    # plot 1: f(x) progress (log scale)
    ax1 = plt.subplot(2, 3, 1)
    ax1.semilogy(range(1, len(f_vals)+1), f_vals, 'o-', linewidth=2, markersize=10, 
                 color='steelblue', markeredgecolor='navy', markeredgewidth=1.5)
    ax1.set_xlabel('experiment number', fontsize=11)
    ax1.set_ylabel('f(x) [log scale]', fontsize=11)
    ax1.set_title('optimization progress', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # add threshold lines
    ax1.axhline(y=2.0, color='green', linestyle='--', alpha=0.4, linewidth=1.5, label='5 pts')
    ax1.axhline(y=1e-6, color='gold', linestyle='--', alpha=0.4, linewidth=1.5, label='6 pts')
    ax1.legend(fontsize=9)
    
    # annotate best
    best_idx = np.argmin(f_vals)
    ax1.plot(best_idx+1, f_vals[best_idx], 'r*', markersize=20, label='best')
    ax1.annotate(f'{f_vals[best_idx]:.2e}', 
                xy=(best_idx+1, f_vals[best_idx]),
                xytext=(15, 15), textcoords='offset points',
                bbox=dict(boxstyle='round', fc='yellow', alpha=0.8),
                arrowprops=dict(arrowstyle='->', lw=2))
    
    # plot 2: f(x) over time
    ax2 = plt.subplot(2, 3, 2)
    ax2.semilogy(dates, f_vals, 's-', linewidth=2, markersize=8, 
                 color='orangered', alpha=0.8)
    ax2.set_xlabel('date/time', fontsize=11)
    ax2.set_ylabel('f(x) [log scale]', fontsize=11)
    ax2.set_title('progress timeline', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right', fontsize=8)
    
    # plot 3: evaluations used
    ax3 = plt.subplot(2, 3, 3)
    colors = ['green' if e <= 100000 else 'red' for e in evals]
    bars = ax3.bar(range(1, len(evals)+1), evals, color=colors, alpha=0.7, edgecolor='black')
    ax3.axhline(y=100000, color='red', linestyle='--', alpha=0.5, linewidth=2, label='budget limit')
    ax3.set_xlabel('experiment number', fontsize=11)
    ax3.set_ylabel('evaluations used', fontsize=11)
    ax3.set_title('budget usage', fontsize=12, fontweight='bold')
    ax3.legend(fontsize=9)
    ax3.grid(True, alpha=0.3, axis='y')
    
    # plot 4: execution time
    ax4 = plt.subplot(2, 3, 4)
    ax4.bar(range(1, len(times)+1), times, color='purple', alpha=0.6, edgecolor='black')
    ax4.set_xlabel('experiment number', fontsize=11)
    ax4.set_ylabel('time (seconds)', fontsize=11)
    ax4.set_title('execution time', fontsize=12, fontweight='bold')
    ax4.grid(True, alpha=0.3, axis='y')
    
    # plot 5: improvement over baseline
    ax5 = plt.subplot(2, 3, 5)
    if len(f_vals) > 1:
        baseline = f_vals[0]
        improvements = [(baseline - f) / baseline * 100 for f in f_vals]
        colors_imp = ['green' if i > 0 else 'red' for i in improvements]
        ax5.bar(range(1, len(improvements)+1), improvements, color=colors_imp, alpha=0.7)
        ax5.axhline(y=0, color='black', linewidth=1)
        ax5.set_xlabel('experiment number', fontsize=11)
        ax5.set_ylabel('% improvement vs baseline', fontsize=11)
        ax5.set_title('relative improvement', fontsize=12, fontweight='bold')
        ax5.grid(True, alpha=0.3, axis='y')
    
    # plot 6: comments log
    ax6 = plt.subplot(2, 3, 6)
    ax6.axis('off')
    ax6.set_title('experiment log', fontsize=12, fontweight='bold', pad=10)
    
    # show all experiments
    y_pos = 0.98
    for i, r in enumerate(results):
        dt = datetime.fromisoformat(r['timestamp']).strftime('%m/%d %H:%M')
        
        # color by performance
        if r['f_value'] < 1e-6:
            color = 'gold'
        elif r['f_value'] < 2.0:
            color = 'lightgreen'
        elif r['f_value'] < 12.0:
            color = 'lightblue'
        else:
            color = 'lightgray'
        
        # format text
        text = f"#{i+1} [{dt}] f={r['f_value']:.2e}\n{r['comment'][:50]}"
        
        ax6.text(0.02, y_pos, text, transform=ax6.transAxes,
                fontsize=8, verticalalignment='top',
                bbox=dict(boxstyle='round,pad=0.4', facecolor=color, alpha=0.6))
        
        y_pos -= 0.12
        if y_pos < 0:
            break
    
    plt.suptitle(f'Optimization Experiments Database ({len(results)} runs)', 
                fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig('results_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\nvisualization saved to 'results_comparison.png'")

## Detailed View of Best Experiment

In [ ]:
# show details of best experiment
if results:
    best_idx = np.argmin([r['f_value'] for r in results])
    best = results[best_idx]
    
    print("=" * 60)
    print(f"BEST EXPERIMENT (#{best_idx+1})")
    print("=" * 60)
    
    print(f"\ndate: {best['timestamp']}")
    print(f"f(x): {best['f_value']:.15e}")
    print(f"evaluations: {best['evaluations']:,}")
    print(f"time: {best['time_seconds']:.2f} seconds")
    print(f"seed: {best['seed']}")
    print(f"\ncomment: {best['comment']}")
    
    print(f"\nsolution vector:")
    sol = np.array(best['solution'])
    print("block 1:", sol[0:5])
    print("block 2:", sol[5:10])
    print("block 3:", sol[10:15])

## Compare Experiments

In [ ]:
# compare specific experiments
if len(results) >= 2:
    print("select experiments to compare (enter experiment numbers):")
    print(f"available: 1 to {len(results)}")
    print("\nlatest experiments:")
    
    for i in range(max(0, len(results)-5), len(results)):
        r = results[i]
        dt = datetime.fromisoformat(r['timestamp']).strftime('%m/%d %H:%M')
        print(f"  #{i+1}: f={r['f_value']:.3e}, {r['comment'][:30]}")
    
    # compare last two automatically
    if len(results) >= 2:
        print(f"\nautomatic comparison: last two experiments")
        print("-" * 60)
        
        prev = results[-2]
        curr = results[-1]
        
        improvement = (prev['f_value'] - curr['f_value']) / prev['f_value'] * 100
        
        print(f"\nprevious: f={prev['f_value']:.6e}")
        print(f"  {prev['comment']}")
        
        print(f"\ncurrent: f={curr['f_value']:.6e}")
        print(f"  {curr['comment']}")
        
        print(f"\nchange: ", end='')
        if improvement > 0:
            print(f"{improvement:.2f}% better ✓")
        elif improvement < 0:
            print(f"{abs(improvement):.2f}% worse ✗")
        else:
            print("same")

## Export Best Result for Submission

In [ ]:
# export best result in submission format
if results:
    best_idx = np.argmin([r['f_value'] for r in results])
    best = results[best_idx]
    
    print("=" * 60)
    print("BEST RESULT FOR SUBMISSION")
    print("=" * 60)
    
    sol = np.array(best['solution'])
    
    print(f"\nXopt = [")
    for i in range(0, 15, 5):
        vals = [f"{sol[j]:.10f}" for j in range(i, min(i+5, 15))]
        sep = "," if i < 10 else ""
        print(f"  {', '.join(vals)}{sep}")
    print("]")
    
    print(f"\nf(Xopt) = {best['f_value']:.15e}")
    print(f"Seed = {best['seed']}")
    print(f"\nAlgorithm: Hybrid IPOP-CMA-ES with Nelder-Mead refinement")
    print(f"Configuration: {best['comment']}")